In [40]:
import pandas as pd
import numpy as np
import os

import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon
import rtree
import pygeos
import fiona
from pykml import parser


pd.options.display.max_columns = 999

In [41]:
os.getcwd()

'/files/ascollanto/01 BARRM DATA CHALLENGE'

# Administrative Bounds

In [42]:
for_dist = 3857 
# 32633
# 6933 equal area


In [43]:
brgy_bounds = gpd.read_file("1-Geodata/1-ADMINISTRATIVE/Administrative Boundaries - BARMM Barangays ao 20190206 (PSA 2016)/AdministrativeBoundariesBARMMBarangays20190206PSA2016.shp")
brgy_bounds.columns = ['PSGC_REGION','REGION_NAME',
                        'PSGC_PROV','PROV_NAME',
                        'PSGC_MUNI','MUNI_NAME',
                        'PSGC_BRGY','BRGY_NAME',
                        'geometry']
brgy_bounds["geometry"] = brgy_bounds['geometry'].to_crs(epsg=for_dist)
brgy_bounds["brgy_area"] = brgy_bounds['geometry'].area/ 10**6

brgy_bounds["geometry"] = brgy_bounds['geometry'].to_crs(epsg=4326)

brgy_bounds['BRGY_NAME'] = brgy_bounds['BRGY_NAME'].str.upper()
brgy_bounds['MUNI_NAME'] = brgy_bounds['MUNI_NAME'].str.upper()
brgy_bounds['PROV_NAME'] = brgy_bounds['PROV_NAME'].str.upper()

# brgy_bounds2 = brgy_bounds[['PSGC_BRGY','geometry']]
# brgy_bounds2.geometry = brgy_bounds2.representative_point()
# brgy_bounds2
brgy_bounds.head()

,PSGC_REGION,REGION_NAME,PSGC_PROV,PROV_NAME,PSGC_MUNI,MUNI_NAME,PSGC_BRGY,BRGY_NAME,geometry,brgy_area
0,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001001,BALIMBING PROPER,"MULTIPOLYGON (((119.95374 5.07851, 119.95384 5...",15.158874
1,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001002,BATU-BATU (POB.),"POLYGON ((119.88709 5.06919, 119.88709 5.06920...",8.254003
2,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001003,BUAN,"MULTIPOLYGON (((120.06363 5.16131, 120.06365 5...",47.332403
3,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001004,DUNGON,"POLYGON ((120.13585 5.19601, 120.13584 5.19603...",55.353446
4,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001005,LUUK BUNTAL,"MULTIPOLYGON (((119.98776 4.98564, 119.98774 4...",6.286992


In [44]:
muni_bounds = gpd.read_file("1-Geodata/1-ADMINISTRATIVE/Administrative Boundaries - BARMM Municipalities ao 20190206 (PSA 2016)/AdministrativeBoundariesBARMMMunicipalities20190206PSA2016.shp")
muni_bounds.columns = ['PSGC_REGION','REGION_NAME',
                        'PSGC_PROV','PROV_NAME',
                        'PSGC_MUNI','MUNI_NAME',
                        'geometry']

muni_bounds['MUNI_NAME'] = muni_bounds['MUNI_NAME'].str.upper()
muni_bounds['PROV_NAME'] = muni_bounds['PROV_NAME'].str.upper()

muni_bounds["geometry"] = muni_bounds['geometry'].to_crs(epsg=for_dist)
muni_bounds["muni_area"] = muni_bounds['geometry'].area/ 10**6

muni_bounds["geometry"] = muni_bounds['geometry'].to_crs(epsg=4326)

muni_bounds.head()



,PSGC_REGION,REGION_NAME,PSGC_PROV,PROV_NAME,PSGC_MUNI,MUNI_NAME,geometry,muni_area
0,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH153600000,LANAO DEL SUR,PH153630000,WAO,"POLYGON ((124.82051 7.53628, 124.82037 7.53616...",322.353733
1,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH156600000,SULU,PH156619000,OMAR,"MULTIPOLYGON (((121.38063 5.92761, 121.38063 5...",87.579051
2,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH156600000,SULU,PH156609000,PARANG,"MULTIPOLYGON (((120.88153 6.01489, 120.88092 6...",70.019520
3,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH153800000,MAGUINDANAO,PH153807000,DATU ODIN SINSUAT (DINAIG),"POLYGON ((124.29669 6.98930, 124.26466 6.98042...",396.588149
4,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH153600000,LANAO DEL SUR,PH153621000,PIAGAPO,"POLYGON ((124.10884 7.95196, 124.10220 7.95666...",122.861796


In [45]:
prov_bounds = gpd.read_file("1-Geodata/1-ADMINISTRATIVE/Administrative Boundaries - BARMM Provinces ao 20190206 (PSA2016)/AdministrativeBoundariesBARMMProvinces20190206PSA2016.shp")
prov_bounds.columns = ['PSGC_REGION','REGION_NAME',
                        'PSGC_PROV','PROV_NAME',
                        'geometry']

prov_bounds['PROV_NAME'] = prov_bounds['PROV_NAME'].str.upper()

prov_bounds["geometry"] = prov_bounds['geometry'].to_crs(epsg=for_dist)
prov_bounds["prov_area"] = prov_bounds['geometry'].area/ 10**6

prov_bounds["geometry"] = prov_bounds['geometry'].to_crs(epsg=4326)

prov_bounds.head()

,PSGC_REGION,REGION_NAME,PSGC_PROV,PROV_NAME,geometry,prov_area
0,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH156600000,SULU,"MULTIPOLYGON (((121.53161 6.01777, 121.53161 6...",1542.357688
1,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH153800000,MAGUINDANAO,"MULTIPOLYGON (((124.04225 7.32152, 124.04225 7...",5030.584700
2,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH124700000,COTABATO (NORTH COTABATO),"MULTIPOLYGON (((124.68982 6.95886, 124.68943 6...",554.957396
3,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH153600000,LANAO DEL SUR,"MULTIPOLYGON (((124.21874 7.89719, 124.21878 7...",3836.238356
4,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH157000000,TAWI-TAWI,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4...",1171.211004


# Floods

In [64]:
flood = gpd.read_file("1-Geodata/4-HAZARDS/Hazards - Flood-prone Areas in BCT (MGB, 2015)/Hazards - Flood - BCT - MGB.shp")

flood["geometry"] = flood['geometry'].to_crs(epsg=for_dist)
flood["flood_area"] = flood['geometry'].area/ 10**6

flood["geometry"] = flood['geometry'].to_crs(epsg=4326)

flood = sjoin(flood,brgy_bounds[['PSGC_BRGY','PSGC_MUNI','PSGC_PROV','geometry']],how='left',op='intersects').drop(columns = ['index_right'])

flood2 = flood[['OBJECTID','geometry']].drop_duplicates()
flood2.geometry = flood2.representative_point()
flood2.columns = ['OBJECTID','geometry_center']
flood2['LONG'] = flood2['geometry_center'].x
flood2['LAT'] = flood2['geometry_center'].y

flood = flood.merge(flood2, on = 'OBJECTID', how = 'left')
flood['TYPE'] = 'Hazards'
flood['SUBTYPE'] = 'Flood'
flood['NAME'] = flood['FloodSusc']
flood['ID'] = 'Flood_'+flood['OBJECTID'].astype(str)
flood['SHAPE_SIZE'] = flood['flood_area']
flood['SHAPE_METRICS'] = 'km2'

flood_viz = flood[['TYPE','SUBTYPE','ID','NAME','LAT','LONG','PSGC_BRGY','PSGC_MUNI','PSGC_PROV','SHAPE_SIZE','SHAPE_METRICS']]
flood_viz.to_csv('Processed/Viz_BARMM_Hazards_Floods.csv')

del flood2
flood.head()

/opt/envs/airflow/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: PROJCS["Transverse_Mercator",GEOGCS["GCS_Clarke 18 ...
Right CRS: EPSG:32651

  


,OBJECTID,QuadName,FloodSusc,SheetNo,SHAPE_Leng,SHAPE_Area,geometry,flood_area,PSGC_BRGY,PSGC_MUNI,PSGC_PROV,geometry_center,LONG,LAT,TYPE,SUBTYPE,NAME,ID,SHAPE_SIZE,SHAPE_METRICS
0,8921,None,High,None,22125.470322,4.555819e+06,"POLYGON ((123.85325 7.69985, 123.85332 7.69960...",0.069407,PH153635037,PH153635000,PH153600000,POINT (123.85123 7.69991),123.851234,7.699908,Hazards,Flood,High,Flood_8921,0.069407,km2
1,8927,None,High,None,2880.620151,3.151018e+05,"POLYGON ((123.87393 7.70191, 123.87377 7.70191...",0.323156,PH153635028,PH153635000,PH153600000,POINT (123.87550 7.70601),123.875497,7.706014,Hazards,Flood,High,Flood_8927,0.323156,km2
2,8930,None,High,None,15658.205523,2.936576e+06,"POLYGON ((123.94836 7.70075, 123.94826 7.70112...",2.987605,PH153635041,PH153635000,PH153600000,POINT (123.94080 7.69644),123.940795,7.696436,Hazards,Flood,High,Flood_8930,2.987605,km2
3,8930,None,High,None,15658.205523,2.936576e+06,"POLYGON ((123.94836 7.70075, 123.94826 7.70112...",2.987605,PH153635024,PH153635000,PH153600000,POINT (123.94080 7.69644),123.940795,7.696436,Hazards,Flood,High,Flood_8930,2.987605,km2
4,8937,None,High,None,104.717980,8.638099e+01,"POLYGON ((123.97010 7.66622, 123.97041 7.66657...",0.000089,PH153635050,PH153635000,PH153600000,POINT (123.97032 7.66646),123.970322,7.666456,Hazards,Flood,High,Flood_8937,0.000089,km2


In [57]:
flood.OBJECTID.nunique()

120

# Fault Line

In [58]:
fault_line = gpd.read_file('1-Geodata/4-HAZARDS/Hazards - Faultlines in ARMM (PhiVolcs, 2018)/ARMM_Faultlines_UTM51N_PhiVolcs.shp')
fault_line['ID'] = 'Fault_'+fault_line.index.astype(str)

fault_line["geometry"] = fault_line['geometry'].to_crs(epsg=32633)
fault_line["fault_length"] = fault_line['geometry'].length/ 10**6

fault_line["geometry"] = fault_line['geometry'].to_crs(epsg=4326)

fault_line = sjoin(fault_line,brgy_bounds[['PSGC_BRGY','PSGC_MUNI','PSGC_PROV','geometry']],how='left',op='intersects').drop(columns = ['index_right'])

fault_line2 = fault_line[['ID','geometry']]
fault_line2.geometry = fault_line2.representative_point()
fault_line2.columns = ['ID','geometry_center']
fault_line2['LONG'] = fault_line2['geometry_center'].x
fault_line2['LAT'] = fault_line2['geometry_center'].y

fault_line = fault_line.merge(fault_line2, on = 'ID', how = 'left')
fault_line['TYPE'] = 'Hazards'
fault_line['SUBTYPE'] = 'Fault Line'
fault_line['NAME'] = fault_line['DESC_']
fault_line['SHAPE_SIZE'] = fault_line['fault_length']
fault_line['SHAPE_METRICS'] = 'km'

fault_viz = fault_line[['TYPE','SUBTYPE','ID','NAME','LAT','LONG','PSGC_BRGY','PSGC_MUNI','PSGC_PROV','SHAPE_SIZE','SHAPE_METRICS']]

del fault_line2
fault_line.head()

/opt/envs/airflow/lib/python3.6/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,OBJECTID,Id,NAME,DESC_,Shape_Leng,geometry,ID,fault_length,PSGC_BRGY,PSGC_MUNI,PSGC_PROV,geometry_center,LONG,LAT,TYPE,SUBTYPE,SHAPE_SIZE,SHAPE_METRICS
0,3,0,Trench,Trench,298585.876045,"LINESTRING (118.51238 0.00004, 118.51238 0.000...",Fault_0,1.228473e-05,NaN,NaN,NaN,POINT (118.51237 0.00005),118.512368,0.000050,Hazards,Fault Line,1.228473e-05,km
1,4,0,Collision zone,Collision zone,50092.723774,"LINESTRING (118.51237 0.00006, 118.51237 0.000...",Fault_1,2.063246e-06,NaN,NaN,NaN,POINT (118.51237 0.00006),118.512366,0.000064,Hazards,Fault Line,2.063246e-06,km
2,13,0,Trace approximate,Trace approximate,37496.352170,"LINESTRING (118.51238 0.00005, 118.51237 0.000...",Fault_2,1.543459e-06,NaN,NaN,NaN,POINT (118.51237 0.00006),118.512374,0.000056,Hazards,Fault Line,1.543459e-06,km
3,76,0,Trace approximate,Trace approximate,9474.239050,"LINESTRING (118.51235 0.00007, 118.51235 0.00006)",Fault_3,3.904518e-07,NaN,NaN,NaN,POINT (118.51235 0.00007),118.512351,0.000065,Hazards,Fault Line,3.904518e-07,km
4,77,0,Active Fault,Active Fault,20528.987986,"LINESTRING (118.51235 0.00007, 118.51235 0.000...",Fault_4,8.457752e-07,NaN,NaN,NaN,POINT (118.51235 0.00007),118.512353,0.000066,Hazards,Fault Line,8.457752e-07,km


In [59]:
fault_line.shape

(42, 18)

# Storm Surge

In [60]:
storm_surge = gpd.read_file('1-Geodata/4-HAZARDS/Hazards - Storm surge-prone areas in ARMM (UP NOAH, 2018)/Storm Surge_UTM51N_NOAH.shp')
storm_surge['ID'] = 'Surge_'+storm_surge.index.astype(str)

storm_surge["geometry"] = storm_surge['geometry'].to_crs(epsg=32633)
storm_surge["surge_area"] = storm_surge['geometry'].area/ 10**6

storm_surge["geometry"] = storm_surge['geometry'].to_crs(epsg=4326)

storm_surge = sjoin(storm_surge,brgy_bounds[['PSGC_BRGY','PSGC_MUNI','PSGC_PROV','geometry']],how='left',op='intersects').drop(columns = ['index_right'])


storm_surge2 = storm_surge[['ID','geometry']]
storm_surge2.geometry = storm_surge2.representative_point()
storm_surge2.columns = ['ID','geometry_center']
storm_surge2['LONG'] = storm_surge2['geometry_center'].x
storm_surge2['LAT'] = storm_surge2['geometry_center'].y

storm_surge = storm_surge.merge(storm_surge2, on = 'ID', how = 'left')
storm_surge['TYPE'] = 'Hazards'
storm_surge['SUBTYPE'] = 'Storm Surge'
storm_surge['NAME'] = storm_surge['HAZ']
storm_surge['SHAPE_SIZE'] = storm_surge['surge_area']
storm_surge['SHAPE_METRICS'] = 'km2'

surge_viz = storm_surge[['TYPE','SUBTYPE','ID','NAME','LAT','LONG','PSGC_BRGY','PSGC_MUNI','PSGC_PROV','SHAPE_SIZE','SHAPE_METRICS']]

# del storm_surge2
storm_surge.head()


/opt/envs/airflow/lib/python3.6/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,HAZ,layer,path,geometry,ID,surge_area,PSGC_BRGY,PSGC_MUNI,PSGC_PROV,geometry_center,LONG,LAT,TYPE,SUBTYPE,NAME,SHAPE_SIZE,SHAPE_METRICS
0,1.0,Basilan_Akbar_StormSurge_SSA4,D:/Geoserver_2015Feb29/Storm Surge Hazards/SSA...,"MULTIPOLYGON (((118.51235 0.00006, 118.51235 0...",Surge_0,1.187559e-10,NaN,NaN,NaN,POINT (118.51235 0.00006),118.512354,0.000060,Hazards,Storm Surge,1.0,1.187559e-10,km2
1,2.0,Basilan_Akbar_StormSurge_SSA4,D:/Geoserver_2015Feb29/Storm Surge Hazards/SSA...,"MULTIPOLYGON (((118.51235 0.00006, 118.51235 0...",Surge_1,2.886954e-10,NaN,NaN,NaN,POINT (118.51235 0.00006),118.512354,0.000060,Hazards,Storm Surge,2.0,2.886954e-10,km2
2,3.0,Basilan_Akbar_StormSurge_SSA4,D:/Geoserver_2015Feb29/Storm Surge Hazards/SSA...,"MULTIPOLYGON (((118.51235 0.00006, 118.51235 0...",Surge_2,3.031916e-09,NaN,NaN,NaN,POINT (118.51235 0.00006),118.512354,0.000060,Hazards,Storm Surge,3.0,3.031916e-09,km2
3,1.0,Basilan_AlBarka_StormSurge_SSA4,D:/Geoserver_2015Feb29/Storm Surge Hazards/SSA...,"MULTIPOLYGON (((118.51235 0.00006, 118.51235 0...",Surge_3,4.227498e-11,NaN,NaN,NaN,POINT (118.51235 0.00006),118.512354,0.000058,Hazards,Storm Surge,1.0,4.227498e-11,km2
4,2.0,Basilan_AlBarka_StormSurge_SSA4,D:/Geoserver_2015Feb29/Storm Surge Hazards/SSA...,"MULTIPOLYGON (((118.51235 0.00006, 118.51235 0...",Surge_4,1.471150e-10,NaN,NaN,NaN,POINT (118.51235 0.00006),118.512354,0.000058,Hazards,Storm Surge,2.0,1.471150e-10,km2


In [61]:
storm_surge.shape

(198, 17)

In [62]:
hazards = pd.concat([flood_viz,surge_viz,fault_viz])
hazards.to_csv('Processed/Viz_BARMM_Hazards.csv')

In [63]:
hazards.shape

(1639, 11)